In [34]:
run_imp = '''
addi    sp, sp, -2032
sd      ra, 2024(sp)                    # 8-byte Folded
sd      s0, 2016(sp)                    # 8-byte Folded
lui     a1, 32
addiw   a1, a1, -2000
sub     sp, sp, a1
mv      s0, a0
addi    a0, sp, 16
addi    a2, zero, 120
mv      a1, s0
call    memcpy@plt
addi    a0, sp, 16
call    run_imp_actual(long*)
addi    a1, sp, 16
addi    a2, zero, 120
mv      a0, s0
call    memcpy@plt
lui     a0, 32
addiw   a0, a0, -2000
add     sp, sp, a0
ld      s0, 2016(sp)                    # 8-byte Folded Reload
ld      ra, 2024(sp)                    # 8-byte Folded Reload
addi    sp, sp, 2032
ret
'''



def print_riscv_instruction(instruction):
    op,i0,i1 = instruction[:3] #assigns the instruction components
    output = instruction[3].strip('\n\t')
    binaryops = {'Plus':'ADD','Minus':'SUB','Times':'MUL','Greater':'SGT','And':'AND','Or':'OR'}
    jumpops = {'JumpIfZero':'BEQZ','JumpIfNonzero':'BNEZ'}
    
    stack = lambda i : str(8*int(i)) + '(a0)' #converts stack index 
    load_stack = lambda i, a: "LD "+a+", "+ stack(i) + "\n\t"
    save_stack = lambda i, a: "SD "+a+", "+ stack(i) + "\n\t"  #save register a to ith index in stack
    binaryop = lambda out_reg, ai, aj, opname: opname + " " + out_reg + ", " + ai +", " + aj + "\n\t" #perform operation a1 op a2, put result in output register
    unaryop = lambda arg1, arg2, opname: opname + " " + arg1 + ", " + arg2 + '\n\t'
    
    
    
    def check_variable1(var):
        register_numb = lambda i : str(int(i) + 1)
        if int(var) > 10:
            return load_stack(var, 'a1')
        else:
            current_register = "s"+register_numb(var)
            save_register = "LD " +"s"+register_numb(var)+ " " + stack(var) + "\n\t"
            return save_register
    
    def check_variable2(var):
        register_numb = lambda i : str(int(i) + 1)
        if int(var) > 10:
            return load_stack(var, 'a2')
        else:
            save_register = "LD " +"s"+register_numb(var)+ " " + stack(var) + "\n\t"
            return save_register
        
    def check_binary_op(var, operator):
        register1_numb = lambda i : str(int(i) + 1)
        register2_numb = lambda i : str(int(i) + 2)
        register3_numb = lambda i : str(int(i) + 3)
        if int(var) > 10:
            return binaryop('a1','a1','a2', operator)
        else:
            return binaryop("s"+register3_numb(var), "s"+register1_numb(var), "s"+register2_numb(var), operator)
        
    def check_unary_opGE(var, operator):
        register1_numb = lambda i : str(int(i) + 1)
        register2_numb = lambda i : str(int(i) + 3)
        if int(var) > 10:
            return unaryop('a3','a1',operator)
        else:
            return unaryop("s"+register2_numb(var), "s"+register1_numb(var), operator)
        
    def check_binary_opE(var):
        register1_numb = lambda i : str(int(i) + 1)
        register2_numb = lambda i : str(int(i) + 2)
        register3_numb = lambda i : str(int(i) + 3)
        register4_numb = lambda i : str(int(i) + 4)
        
        if int(var) > 10:
            instruction_set = binaryop('a3','a1','a2','SLT') + \
            binaryop('a4','a1','a2','SGT') + \
            binaryop('a2','a3','a4','XOR') + \
            unaryop('a1','a2','NOT')
            return instruction_set
        else:
            instruction_set = binaryop("s"+register3_numb(var),"s"+register1_numb(var),"s"+register2_numb(var),'SLT') + \
            binaryop("s"+register4_numb(var),"s"+register1_numb(var),"s"+register2_numb(var),'SGT') + \
            binaryop("s"+register2_numb(var),"s"+register3_numb(var),"s"+register4_numb(var),'XOR') + \
            unaryop("s"+register1_numb(var),"s"+register2_numb(var),'NOT')
            return instruction_set
        
    def check_unary_opNOT(var, operator):
        register1_numb = lambda i : str(int(i) + 1)
        if int(var) > 10:
            return unaryop('a1','a1',operator)
        else:
            return unaryop("s"+register1_numb(var), "s"+register1_numb(var), operator)
        
    def check_unary_opLI(var, operator):
        register1_numb = lambda i : str(int(i) + 1)
        if int(var) > 10:
            return unaryop('a1',var,operator)
        else:
            return unaryop("s"+register1_numb(var), var, operator)
        
    def store_output(var): 
        register_numb = lambda i : str(int(i) + 1)
        if int(var) > 10:
            return save_stack(var, 'a1')
        else:
            save_register = "SD " +"s"+register_numb(var)+ " " + stack(var) + "\n\t"
            return save_register
    
    if op in binaryops: 
        riscv = check_variable1(i0) + \
        check_variable2(i1) + \
        check_binary_op(i0, binaryops[op]) + \
        store_output(output)
        
    
    elif op in jumpops:
        riscv = check_variable1(i0) + \
        unaryop('a1',i1,jumpops[op])
        
    elif op == 'GreaterEq': #a3 = (a1 < a2), then a1 = not a3, save a1 to stack output
        riscv = check_variable1(i0) + \
        check_variable2(i1) + \
        check_binary_op(i0, 'SLT') + \
        check_unary_opGE(i0, 'NOT') + \
        store_output(output)
        
    elif op == 'Equal': #a3 = (a1 < a2), a4 = (a1 > a2), a2 = (a3 xor a4), a1 = not a2, save a1 to stack output
        riscv = check_variable1(i0) + \
        check_variable2(i1) + \
        check_binary_opE(i0) + \
        store_output(output)
        
    elif op == 'Not': 
        riscv = check_variable1(i0) + \
        check_unary_opNOT(i0,'NOT') + \
        store_output(output)
        
    elif op == 'LoadConstant':
        riscv = check_unary_opLI(i0,'LI') + \
        store_output(output)
        
    elif op == 'Label':
        riscv = i0 + '\n\t'
        
    elif op == 'Move':
        riscv = check_variable1(i0) + \
        store_output(output)
        
    elif op == 'Jump':
        riscv = 'JAL x0, ' + i0
        
    else: assert False, 'operation not found'
    
    return riscv

def print_run_imp_actual(instructions):
    run_imp_actual = 'run_imp_actual(long*):\n\t'
    for i in instructions:
        run_imp_actual+=print_riscv_instruction(i)
    return run_imp_actual
    

In [35]:
with open('../example_op_file.txt', 'r') as f:
    operations = f.readlines()
    instructions = [o.split(' ') for o in operations]

# #Printing each intermediate instruction and its riscv translation
# for i in instructions:
#     op,i0,i1 = i[:3] #assigns the instruction components
#     output = i[3].strip('\n\t')
#     print('instruction: ',op,i0,i1,output)
#     print('\t'+print_riscv_instruction(i))

print(print_run_imp_actual(instructions))
print(run_imp)

run_imp_actual(long*):
	LD s2 8(a0)
	LD s3 16(a0)
	ADD s4, s2, s3
	SD s4 24(a0)
	LD s2 8(a0)
	LD s3 16(a0)
	SUB s4, s2, s3
	SD s4 24(a0)
	LD s2 8(a0)
	LD s3 16(a0)
	MUL s4, s2, s3
	SD s4 24(a0)
	LD s2 8(a0)
	LD s3 16(a0)
	SGT s4, s2, s3
	SD s4 24(a0)
	LD s2 8(a0)
	LD s3 16(a0)
	SLT s4, s2, s3
	NOT s4, s2
	SD s4 24(a0)
	LD s2 8(a0)
	LD s3 16(a0)
	SLT s4, s2, s3
	SGT s5, s2, s3
	XOR s3, s4, s5
	NOT s2, s3
	SD s4 24(a0)
	LD s2 8(a0)
	LD s3 16(a0)
	OR s4, s2, s3
	SD s4 24(a0)
	LD s2 8(a0)
	LD s3 16(a0)
	AND s4, s2, s3
	SD s4 24(a0)
	LD s2 8(a0)
	NOT s2, s2
	SD s4 24(a0)
	LI s2, 1
	SD s4 24(a0)
	LD s2 8(a0)
	BEQZ a1, .LBB_2
	LD s2 8(a0)
	BNEZ a1, .LBB_2
	.LBB_1
	LD s2 8(a0)
	SD s4 24(a0)
	JAL x0, .LBB_1

addi    sp, sp, -2032
sd      ra, 2024(sp)                    # 8-byte Folded
sd      s0, 2016(sp)                    # 8-byte Folded
lui     a1, 32
addiw   a1, a1, -2000
sub     sp, sp, a1
mv      s0, a0
addi    a0, sp, 16
addi    a2, zero, 120
mv      a1, s0
call    memcpy@plt
addi    a0,